# Aggregating OpenFace Action Units on every 30 frames
This notebook contains a function `agg_open_face` to aggregate the openface action units on frame basis.

In [179]:
def agg_open_face(file_name):
    import pandas as pd
    
    try:
        data = pd.read_csv('~/Downloads/5-27-2021-h7e9p.csv')
    except:
        print('Specified file does not exists')
        return

    # converting timestamp to pandas timestamp
    data['timestamp'] = pd.to_datetime(data['timestamp'],unit='s')
    
    # columns of interest
    columns_interest= ['frame','total_count',
                   'AU01_c','AU02_c','AU04_c','AU05_c','AU06_c','AU07_c','AU09_c','AU10_c',
                   'AU12_c','AU14_c','AU15_c','AU17_c','AU20_c','AU23_c','AU25_c',
                   'AU26_c','AU45_c','AU05_c','AU06_c','AU07_c','AU09_c','AU10_c',
                  ]
    
    # new dataframe to store aggregated results
    ag_frame = pd.DataFrame(columns = columns_interest)
    
    # activity start and end timestamp
    activity_start = data['timestamp'][0]
    activity_end = data['timestamp'][data.shape[0]-1]
    
    # Window size: here we are using 30 milli. That means if we consider first window than it consider 
    # withing 0.000 to 0.030 timestamp. Open face actually stores frame number in sequence
    frame_start = activity_start
    frame_end = frame_start + pd.Timedelta('30milli')
    
    

    frame_no = 1
    while frame_end <= activity_end:
        #print('Frame:',frame_no)
        #print('  Start:',frame_start)  
        #print('  End:',frame_end)
        
        
    
        # mask to extract data of window
        mask = (data['timestamp'] < frame_end) & (data['timestamp'] >= frame_start)
        frame_df = data.loc[mask]
        
        # create dictionary to store aggregated data for each window
        tmp = {'frame':frame_no,'total_count':frame_df.shape[0]}
        
        # aggregate data for each column (here we are using sum as every action unit value is either 0 or 1)
        for col in columns_interest[2:]:
            tmp[col] = sum(frame_df[col].tolist())
    
        # appending aggregated record
        ag_frame = ag_frame.append(tmp,ignore_index=True)
        
        # shifting window
        frame_start = frame_end
        frame_end = frame_start + pd.Timedelta('30milli')
        frame_no += 1
        
    return ag_frame


### How to use
You can simply call the above function. This function takes the path to your openface csv file.
#### Example

In [180]:
result = agg_open_face('/Users/pankaj/Downloads/5-27-2021-h7e9p.csv')

In [182]:
result

,frame,total_count,AU01_c,AU02_c,AU04_c,AU05_c,AU06_c,AU07_c,AU09_c,AU10_c,...,AU20_c,AU23_c,AU25_c,AU26_c,AU45_c,AU05_c,AU06_c,AU07_c,AU09_c,AU10_c
0,1.0,30.0,0.0,0.0,0.0,30.0,0.0,0.0,0.0,22.0,...,22.0,12.0,15.0,11.0,9.0,30.0,0.0,0.0,0.0,22.0
1,2.0,30.0,0.0,0.0,0.0,30.0,0.0,0.0,0.0,30.0,...,28.0,0.0,0.0,0.0,0.0,30.0,0.0,0.0,0.0,30.0
2,3.0,30.0,0.0,0.0,0.0,30.0,0.0,0.0,0.0,24.0,...,7.0,0.0,0.0,8.0,0.0,30.0,0.0,0.0,0.0,24.0
3,4.0,30.0,0.0,0.0,0.0,30.0,0.0,0.0,0.0,6.0,...,8.0,18.0,0.0,25.0,0.0,30.0,0.0,0.0,0.0,6.0
4,5.0,30.0,0.0,0.0,0.0,30.0,0.0,0.0,0.0,15.0,...,0.0,10.0,0.0,5.0,0.0,30.0,0.0,0.0,0.0,15.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3113,3114.0,30.0,0.0,0.0,0.0,30.0,0.0,0.0,0.0,30.0,...,30.0,0.0,30.0,26.0,0.0,30.0,0.0,0.0,0.0,30.0
3114,3115.0,30.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.0,...,27.0,0.0,23.0,0.0,0.0,0.0,0.0,0.0,0.0,17.0
3115,3116.0,30.0,0.0,0.0,0.0,21.0,0.0,0.0,0.0,0.0,...,11.0,0.0,22.0,0.0,0.0,21.0,0.0,0.0,0.0,0.0
3116,3117.0,30.0,0.0,0.0,0.0,30.0,0.0,0.0,0.0,0.0,...,0.0,0.0,30.0,0.0,0.0,30.0,0.0,0.0,0.0,0.0


You can use the aggregated data to see whether a particular action unit was present in a window or not.
Each aggregated window has a total_count measure giving the number of total records in a particular window.
You can divide the value under any action unit by this number and if this number is higher than .5 that means that action unit was present more than high time of window.

#### Mapping action unit to emotion
You can use rules from following research paper to map action units to emotion.
```
S. Velusamy, H. Kannan, B. Anand, A. Sharma and B. Navathe, 
"A method to infer emotions from facial Action Units," 
2011 IEEE International Conference on Acoustics, Speech and Signal Processing (ICASSP), 2011, pp. 2028-2031, 
doi: 10.1109/ICASSP.2011.5946910.
```


In [ ]:
# The mapping of AU to emotion is used from following reference
"""
S. Velusamy, H. Kannan, B. Anand, A. Sharma and B. Navathe, 
"A method to infer emotions from facial Action Units," 
2011 IEEE International Conference on Acoustics, Speech and Signal Processing (ICASSP), 2011, pp. 2028-2031, 
doi: 10.1109/ICASSP.2011.5946910.

We deleted  surprise emotion because it require AU27 which is not provided by OpenFace.
"""
emotion={
    'angry':['AU23_c','AU07_r','AU17_r','AU04_r','AU02_r'],
    'fear':['AU20_c','AU04_r','AU01_r','AU05_r','AU07_r'],
    'sad':['AU15_c','AU01_r','AU04_r','AU17_r','AU10_r'],
    'happy':['AU12_c','AU06_r','AU26_r','AU10_r','AU23_r'],
    #'surprise':['AU27_c','AU02_r','AU01_r','AU05_r','AU26_r'],
    'disgust':['AU09_c','AU07_r','AU04_r','AU17_r','AU06_r'],
        }